In [ ]:
!pip install --upgrade diffusers transformers accelerate safetensors pillow


In [ ]:
from google.colab import files

uploaded = files.upload()
input_image_path = list(uploaded.keys())[0]
print("✅ Uploaded:", input_image_path)


Saving img1.jpg to img1 (1).jpg
✅ Uploaded: img1 (1).jpg


In [ ]:
import requests
from PIL import Image
import torch
from diffusers import StableDiffusionInpaintPipeline

def run_multi_variant_product_enhancer(image_path, api_key, output_prefix="variant_output"):

    # STEP 1: Background Removal
    def remove_background(input_path, output_path):
        url = "https://api.rembg.com/rmbg"
        headers = {"x-api-key": api_key}
        files = {"image": open(input_path, "rb")}

        response = requests.post(url, headers=headers, files=files)
        if response.status_code == 200:
            with open(output_path, "wb") as f:
                f.write(response.content)
            print("✅ Background removed")
        else:
            raise Exception(f"Background removal failed: {response.status_code} {response.text}")

    # STEP 2: Square Crop
    def center_crop_to_square(input_path, output_path, size=1024):
        img = Image.open(input_path)
        min_dim = min(img.width, img.height)
        left = (img.width - min_dim) // 2
        top = (img.height - min_dim) // 2
        img = img.crop((left, top, left + min_dim, top + min_dim))
        img = img.resize((size, size), Image.LANCZOS)
        img.save(output_path)
        print("✅ Cropped & resized")

    # STEP 3: Mask Creation
    def create_mask_from_alpha(image_path, mask_path):
        img = Image.open(image_path).convert("RGBA")
        alpha = img.getchannel("A")
        mask = Image.new("L", img.size, 255)
        mask.paste(0, mask=alpha)
        mask.save(mask_path)
        print("✅ Mask created")

    # STEP 4: Inpainting with Multiple Prompts
    def enhance_variants(init_img_path, mask_img_path, output_prefix, prompts):
        pipe = StableDiffusionInpaintPipeline.from_pretrained(
            "runwayml/stable-diffusion-inpainting",
            torch_dtype=torch.float16
        ).to("cuda")

        init_image = Image.open(init_img_path).convert("RGB").resize((512, 512))
        mask_image = Image.open(mask_img_path).convert("L").resize((512, 512))

        for i, prompt in enumerate(prompts):
            result = pipe(prompt=prompt, image=init_image, mask_image=mask_image).images[0]
            filename = f"{output_prefix}_{i+1}.png"
            result.save(filename)
            print(f"✅ Variant {i+1} saved: {filename}")

    # PIPELINE RUN
    cleaned = f"{output_prefix}_cleaned.png"
    cropped = f"{output_prefix}_square.png"
    mask = f"{output_prefix}_mask.png"

    remove_background(image_path, cleaned)
    center_crop_to_square(cleaned, cropped)
    create_mask_from_alpha(cropped, mask)

    # PROMPT VARIANTS – All with white background
    prompts = [
        "a plain white studio background with soft diffused lighting, realistic shadow under the product, centered layout, front-facing view, sharp detail, professional look",
        "studio lighting background",
        "solid plain white background"
    ]

    enhance_variants(cropped, mask, output_prefix, prompts)


In [ ]:
rembg_api_key = "7093fd23-ea8d-40fc-91cd-becaf310d916"  # Replace with your API key
run_multi_variant_product_enhancer(input_image_path, rembg_api_key)


✅ Background removed
✅ Cropped & resized
✅ Mask created


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Variant 1 saved: variant_output_1.png


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Variant 2 saved: variant_output_2.png


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Variant 3 saved: variant_output_3.png


In [ ]:
from google.colab import files

for i in range(1, 4):
    files.download(f"variant_output_{i}.png")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>